# Data Aggregation and Group Operations
Categorizing a dataset and applying a function to each group, whether an aggregation
or transformation, can be a critical component of a data analysis workflow. After
loading, merging, and preparing a dataset, you may need to compute **group statistics**
or possibly **pivot tables** for reporting or visualization purposes. `pandas` provides a
versatile `groupby` interface, enabling you to slice, dice, and summarize datasets in a
natural way.

One reason for the popularity of relational databases and SQL (which stands for
“structured query language”) is the ease with which data can be joined, filtered,
transformed, and aggregated. However, query languages like SQL impose certain
limitations on the kinds of group operations that can be performed.

As you will see, with the expressiveness of Python and `pandas`, we can perform quite **complex** group
operations by expressing them as custom Python functions that manipulate the data
associated with each group. We will learn how to:
- **Split** a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)
- Calculate **group summary statistics**, like count, mean, or standard deviation, or a
user-defined function
- **Apply** within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection
- Compute **pivot tables** and **cross-tabulations**
- Perform **quantile analysis** and other statistical group analyses

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
import numpy as np
import pandas as pd

## How to Think About Group Operations
Hadley Wickham, an author of many popular packages for the R programming
language, coined the term **split-apply-combine** for describing group operations. 

1. **(SPLIT):** In the first stage of the process, data contained in a pandas object, whether a Series, DataFrame, or otherwise, is **split** into groups based on one or more keys that you provide.
The splitting is performed on a particular axis of an object. For example, a DataFrame
can be grouped on its rows (`axis="index"`) or its columns (`axis="columns"`). 
2. **(APPLY):** Once this is done, a function is **applied** to each group, producing a new value. 
3. **(COMBINE):** Finally, the results of all those function applications are **combined** into a result object. The
form of the resulting object will usually depend on what’s being done to the data. 

In the Figure a mockup of a simple group aggregation.
<img src="Img/pd_groupAggregation.png" alt="Illustration of a group aggregation" title="Illustration of a group aggregation" />

Each **grouping key** can take many forms, and the keys do **not** have to be all of the
same type:
- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dictionary or Series giving a correspondence between the values on the axis
being grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index


Note that the latter three methods are shortcuts for producing an array of values to
be used to split up the object. Don’t worry if this all seems abstract.

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


Suppose you wanted to compute the **mean** of the `data1` column using the labels from
`key1`. There are a number of ways to do this. One is to access `data1` and call `groupby`
with the column (a Series) at `key1`

In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

This `grouped` variable is now a special `GroupBy` object. It has **not** actually computed
anything yet except for some *intermediate* data about the group key `df["key1"]`.

The idea is that this object has all of the information needed to then apply some
operation to each of the groups. For example, to compute group **means** we can call
the GroupBy’s `mean` method

In [5]:
grouped.mean()

key1
a    0.555881
b    0.705025
Name: data1, dtype: float64

In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.204708
      2       0.478943
b     1       1.965781
      2      -0.555730
Name: data1, dtype: float64

Here we grouped the data using two keys, and the resulting Series now has a **hierarchical index** consisting of the unique pairs of keys observed

In [7]:
means.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

In [8]:
means.unstack()

key2,1,2
key1,,
a,-0.204708,0.478943
b,1.965781,-0.555730


In this example, the **group keys** are all `Series`, though they could be any arrays of the
right length

In [10]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [9]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005    0.936175
    2006   -0.519439
OH  2005   -0.380219
    2006    1.029344
Name: data1, dtype: float64

In [11]:
df.shape,  states.shape, len(years)

((7, 4), (7,), 7)

Frequently, the grouping information is found in the same DataFrame as the data you
want to work on. In that case, you can pass **column names** (whether those are strings,
numbers, or other Python objects) as the group keys

In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.555881,0.441920
b,1.5,0.705025,-0.144516


In [18]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,0.333636,0.115218
2,-0.038393,0.888106


In [16]:
df.groupby("key2").mean()

TypeError: agg function failed [how->mean,dtype->object]

In [17]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

You may have noticed in the second case, `df.groupby("key2").mean()`, that there is
**no** `key1` column in the result. Because `df["key1"]` is **not** numeric data, it is said to
be a nuisance column, which is therefore automatically excluded from the result. By
default, all of the **numeric** columns are **aggregated**, though it is possible to filter down
to a subset, as you’ll see soon.

Regardless of the objective in using `groupby`, a generally useful GroupBy method is
`size`, which returns a Series containing group sizes

In [19]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

Note that any **missing values** in a group key are **excluded** from the result by default. 
This behavior can be disabled by passing `dropna=False` to groupby

In [20]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [21]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

A group function similar in spirit to `size` is `count`, which computes the number of
**nonnull** values in each group

In [22]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups
The object returned by `groupby` supports **iteration**, generating a sequence of *2-tuples*
containing the group name along with the chunk of data.

In [23]:
for name, group in df.groupby("key1"):
    print("name: ", name)
    print("group:\n", group)
    print()


name:  a
group:
   key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992

name:  b
group:
   key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221



In the case of **multiple keys**, the first element in the tuple will be a tuple of key values

In [24]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print("keys: ",(k1, k2))
    print("group:\n", group)
    print()


keys:  ('a', np.int64(1))
group:
   key1  key2     data1     data2
0    a     1 -0.204708  0.281746

keys:  ('a', np.int64(2))
group:
   key1  key2     data1     data2
1    a     2  0.478943  0.769023

keys:  ('b', np.int64(1))
group:
   key1  key2     data1     data2
4    b     1  1.965781 -1.296221

keys:  ('b', np.int64(2))
group:
   key1  key2    data1     data2
3    b     2 -0.55573  1.007189



Of course, you can choose to do whatever you want with the pieces of data. 

A recipe you may find useful is computing a **dictionary** of the data pieces as a one-liner

In [25]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -0.204708  0.281746
 1    a     2  0.478943  0.769023
 5    a  <NA>  1.393406  0.274992,
 'b':   key1  key2     data1     data2
 3    b     2 -0.555730  1.007189
 4    b     1  1.965781 -1.296221}

In [26]:
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221


By default `groupby` groups on `axis="index"`, but you can group on any of the other
axes. 

For example, we could group the **columns** of our example df here by whether
they start with "key" or "data"

In [27]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/1884927282.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [29]:
df.T.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"})

We can print out the groups like so

In [30]:
for group_key, group_values in grouped:
    print("key: ",group_key)
    print("group\n", group_values)
    print()


key:  data
group
       data1     data2
0 -0.204708  0.281746
1  0.478943  0.769023
2 -0.519439  1.246435
3 -0.555730  1.007189
4  1.965781 -1.296221
5  1.393406  0.274992
6  0.092908  0.228913

key:  key
group
    key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1



### Selecting a Column or Subset of Columns
Indexing a GroupBy object created from a DataFrame with a column name or array
of column names has the effect of column subsetting for aggregation. 

This means that:
```python
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]
```
are conveniences for:
```python
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])
```

Especially for **large datasets**, it may be desirable to **aggregate only a few columns**. For
example, in the preceding dataset, to compute the **means** for just the data2 column
and get the result as a DataFrame, we could write

In [32]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.281746
     2     0.769023
b    1    -1.296221
     2     1.007189

The object returned by this indexing operation is a `grouped` DataFrame if a list or
array is passed, or a grouped Series if only a single column name is passed as a scalar

In [31]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped.mean()

key1  key2
a     1       0.281746
      2       0.769023
b     1      -1.296221
      2       1.007189
Name: data2, dtype: float64

### Grouping with Dictionaries and Series
Grouping information may exist in a form other than an array

In [44]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,0.124121,0.302614,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


In [45]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


Now, suppose I have a group correspondence for the columns and want to sum the
columns by group

In [46]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

Now, you could construct an array from this dictionary to pass to `groupby`, but
instead we can just pass the dictionary (I included the key "f" to highlight that
unused grouping keys are OK)

In [47]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-2.373480,3.908371
Steve,3.725929,-1.999539
Wanda,0.523772,-0.576147
Jill,-3.201385,-1.230495
Trey,-1.146107,-1.364125


The same functionality holds for Series, which can be viewed as a fixed-size mapping

In [48]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [49]:
people.groupby(map_series, axis="columns").count()

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions
Using Python **functions** is a more generic way of defining a group mapping compared
with a dictionary or Series. Any function passed as a group key will be called **once**
**per index value** (or once per column value if using `axis="columns"`), with the
**return values** being used as the **group names**. 

More concretely, consider the example
DataFrame from the previous section, which has people’s first names as index values.
Suppose you wanted to **group by name length**. While you could compute an array of
string lengths, it’s simpler to just pass the len function

In [50]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,0.483052,-0.153399,-2.097088,-2.250405,-2.924273
5,-1.015657,-0.539741,0.476985,3.772716,-1.020287


Mixing functions with arrays, dictionaries, or Series is not a problem, as **everything gets converted to arrays internally**

In [51]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,two,-0.860757,-0.713544,-1.265934,-2.370232,-1.860761
5,one,-0.577087,-0.539741,0.476985,0.523772,-1.021228


### Grouping by Index Levels
A final convenience for **hierarchically indexed** datasets is the ability to aggregate
using one of the levels of an axis index

In [54]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])

columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [55]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.660524  0.862580 -0.010032  0.050009  0.670216
1      0.852965 -0.955869 -0.023493 -2.304234 -0.652469
2     -1.218302 -1.332610  1.074623  0.723642  0.690002
3      1.001543 -0.503087 -0.622274 -0.921169 -0.726213

In [56]:
hier_df.groupby(level="cty", axis="columns").count()

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

**Aggregations** refer to any **data transformation** that produces **scalar values** from arrays. The preceding examples have used several of them, including `mean`, `count`, `min`, and
`sum`.

<img src="Img/pd_groupby.png" alt="Optimized groupby methods" title="Optimized groupby methods" />
<img src="Img/pd_groupby_2.png" alt="Optimized groupby methods" title="Optimized groupby methods" />


You can use **aggregations** of your own devising and additionally call any method
that is also defined on the object being grouped. For example, the `nsmallest`
Series method selects the smallest requested number of values from the data.

While `nsmallest` is **not** explicitly implemented for GroupBy, we can still use it
with a **nonoptimized implementation**. Internally, GroupBy **slices up** the Series, calls
`piece.nsmallest(n)` for each piece, and then assembles those results into the result
object

In [57]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [58]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(1)
# Find the smallest value of data1 within each group defined by key1

key1   
a     0   -0.204708
b     3   -0.555730
Name: data1, dtype: float64

To use your own aggregation functions, pass any function that aggregates an array to
the aggregate method or its short alias agg

In [60]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


You may notice that some methods, like `describe`, also work, even though they are
not aggregations, strictly speaking

In [61]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.555881  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.705025  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.936175  1.393406   3.0  0.441920  0.283299  0.274992  0.278369   
b     1.335403  1.965781   2.0 -0.144516  1.628757 -1.296221 -0.720368   

                                    
           50%       75%       max  
key1                                
a     0.281746  0.525384  0.769023  
b    -0.144516  0.431337  1.007189  

[2 rows x 24 columns]

**Custom** aggregation functions are generally much **slower** than the
optimized functions found in Table 10-1. This is because there
is some extra overhead (function calls, data rearrangement) in
constructing the intermediate group data chunks.

### Column-Wise and Multiple Function Application

In [62]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [63]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


As you’ve already seen, aggregating a Series or all of the columns of a DataFrame is
a matter of using **aggregate** (or agg) with the desired function or calling a method
like `mean` or `std`. However, you may want to aggregate using a different function,
depending on the column, or multiple functions at once. Fortunately, this is possible
to do, which I’ll illustrate through a number of examples. 

First, I’ll group the tips by
day and smoker

In [68]:
grouped = tips.groupby(["day", "smoker"])
for name, group in grouped:
    print("name: ", name)
    print("group:\n", group)
    print()

name:  ('Fri', 'No')
group:
      total_bill   tip smoker  day    time  size   tip_pct
91        22.49  3.50     No  Fri  Dinner     2  0.155625
94        22.75  3.25     No  Fri  Dinner     2  0.142857
99        12.46  1.50     No  Fri  Dinner     2  0.120385
223       15.98  3.00     No  Fri   Lunch     3  0.187735

name:  ('Fri', 'Yes')
group:
      total_bill   tip smoker  day    time  size   tip_pct
90        28.97  3.00    Yes  Fri  Dinner     2  0.103555
92         5.75  1.00    Yes  Fri  Dinner     2  0.173913
93        16.32  4.30    Yes  Fri  Dinner     2  0.263480
95        40.17  4.73    Yes  Fri  Dinner     4  0.117750
96        27.28  4.00    Yes  Fri  Dinner     2  0.146628
97        12.03  1.50    Yes  Fri  Dinner     2  0.124688
98        21.01  3.00    Yes  Fri  Dinner     2  0.142789
100       11.35  2.50    Yes  Fri  Dinner     2  0.220264
101       15.38  3.00    Yes  Fri  Dinner     2  0.195059
220       12.16  2.20    Yes  Fri   Lunch     2  0.180921
221       13

In [65]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If you pass a **list of functions** or function names instead, you get back a DataFrame
with column names taken from the functions

In [70]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Here we passed a list of aggregation functions to `agg` to evaluate independently on the
data groups.

You don’t need to accept the names that GroupBy gives to the columns; notably,
lambda functions have the name "<lambda>", which makes them hard to identify
(you can see for yourself by looking at a function’s __name__ attribute). Thus, if you
pass a **list of (name, function) tuples**, the first element of each tuple will be used
as the DataFrame column names (you can think of a list of 2-tuples as an ordered
mapping)

In [71]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/1734782830.py:1: FutureWarning: The provided callable <function std at 0x105c41800> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a **DataFrame** you have more options, as you can specify a **list of functions
to apply to all of the columns or different functions per column**. 

In [72]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [73]:
# DataFrame has hierarchical columns
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [77]:
# a list of tuples with custom names can be passed
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/90882496.py:3: FutureWarning: The provided callable <function var at 0x105c41940> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

Now, suppose you wanted to apply potentially **different functions to one or more of the columns**. 
To do this, pass a **dictionary** to `agg` that contains a mapping of column
names to any of the function specifications listed so far

In [78]:
grouped.agg({"tip" : np.max, "size" : "sum"})

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2044707828.py:1: FutureWarning: The provided callable <function max at 0x105c40cc0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip" : np.max, "size" : "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [79]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

A DataFrame will have **hierarchical columns** only if **multiple** functions are applied to
at least one column.

### Returning Aggregated Data Without Row Indexes
In all of the examples up until now, the aggregated data comes back with an index,
potentially hierarchical, composed from the unique group key combinations. Since
this isn’t always desirable, you can disable this behavior in most cases by passing
`as_index=False` to groupby

In [80]:
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [83]:
grouped = tips.groupby(["day", "smoker"])
grouped.mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

## Apply: General split-apply-combine
The most general-purpose GroupBy method is `apply`, which is the subject of this
section.

1. **splits** the object being manipulated into pieces, 
2. **invokes** the passed function on each piece, 
3. and then attempts to **concatenate** the pieces.

Returning to the tipping dataset from before, suppose you wanted to select the **top five** 
`tip_pct` values by group. First, write a function that selects the rows with the
largest values in a particular column

In [84]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [85]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [87]:
tips.groupby("smoker").apply(top)

total_bill   tip   day    time  size   tip_pct
smoker                                                    
No     232       11.61  3.39   Sat  Dinner     2  0.291990
       149        7.51  2.00  Thur   Lunch     2  0.266312
       51        10.29  2.60   Sun  Dinner     2  0.252672
       185       20.69  5.00   Sun  Dinner     5  0.241663
       88        24.71  5.85  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15   Sun  Dinner     2  0.710345
       178        9.60  4.00   Sun  Dinner     2  0.416667
       67         3.07  1.00   Sat  Dinner     1  0.325733
       183       23.17  6.50   Sun  Dinner     4  0.280535
       109       14.31  4.00   Sat  Dinner     2  0.279525

In [ ]:
tips.groupby("smoker").apply(top, include_groups=False)

What has happened here? 
- First, the tips DataFrame is **split** into groups based on the
value of smoker. 
- Then the **top** function is called on each group
- The results of each function call are **glued** together using `pandas.concat`, labeling the pieces with
the group names. 

The result therefore has a **hierarchical index** with an inner level that
contains index values from the original DataFrame.

If you pass a function to apply that takes other arguments or keywords, you can pass
these after the function

In [88]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/643835440.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

Beyond these basic usage mechanics, getting the most out of `apply` may require some
creativity. What occurs inside the function passed is up to you; it must either **return a pandas object or a scalar value**.

In [89]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [95]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like describe, it is actually just a
shortcut for:
```python
def f(group):
    return group.describe()
grouped.apply(f)
```

### Suppressing the Group Keys
In the preceding examples, you see that the resulting object has a **hierarchical index
formed from the group keys**, along with the indexes of each piece of the original
object. You can **disable** this by passing `group_keys=False` to groupby

In [96]:
tips.groupby("smoker", group_keys=False).apply(top)

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/3851957478.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [97]:
tips.groupby("smoker").apply(top)

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2530541573.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

### Quantile and Bucket Analysis
As you may recall from Chapter 8, pandas has some tools, in particular `pandas.cut`
and `pandas.qcut`, for slicing data up into buckets with bins of your choosing, or by
sample quantiles. Combining these functions with `groupby` makes it convenient to
perform bucket or quantile analysis on a dataset. 

Consider a simple random dataset
and an equal-length bucket categorization using `pandas.cut`

In [98]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.222896,0.329313
1,0.051316,-0.732599
2,-1.157719,0.137889
3,0.816707,-0.719672
4,0.433610,1.927640


In [99]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0    (-1.23, 0.489]
1    (-1.23, 0.489]
2    (-1.23, 0.489]
3    (0.489, 2.208]
4    (-1.23, 0.489]
5    (0.489, 2.208]
6    (0.489, 2.208]
7    (-1.23, 0.489]
8    (0.489, 2.208]
9    (-1.23, 0.489]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

The `Categorical` object returned by `cut` can be passed directly to `groupby`. So we
could compute a set of group statistics for the quartiles

In [100]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/2015905501.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     94 -1.658156
                data2 -3.399312  1.670835     94 -0.043940
(-1.23, 0.489]  data1 -1.228918  0.488675    599 -0.325323
                data2 -2.989741  3.260383    599 -0.010643
(0.489, 2.208]  data1  0.489965  2.200997    296  1.067500
                data2 -3.745356  2.954439    296  0.092068
(2.208, 3.928]  data1  2.212303  3.927528     11  2.622638
                data2 -1.929776  1.765640     11  0.055158

In [101]:
# Keep in mind the same result could have been computed more simply with
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    94 -1.658156 -3.399312  1.670835    94   
(-1.23, 0.489]  -1.228918  0.488675   599 -0.325323 -2.989741  3.260383   599   
(0.489, 2.208]   0.489965  2.200997   296  1.067500 -3.745356  2.954439   296   
(2.208, 3.928]   2.212303  3.927528    11  2.622638 -1.929776  1.765640    11   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.043940  
(-1.23, 0.489]  -0.010643  
(0.489, 2.208]   0.092068  
(2.208, 3.928]   0.055158

These were **equal-length** buckets; to compute **equal-size** buckets based on sample
quantiles, use `pandas.qcut`. 

We can pass 4 as the number of bucket compute sample quartiles, and pass `labels=False` to obtain just the quartile indices instead of intervals

In [102]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    2
2    0
3    3
4    2
Name: data1, dtype: int64

In [103]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.683066    250 -1.209525
      data2 -3.399312  2.628441    250 -0.047773
1     data1 -0.679400 -0.031599    250 -0.366159
      data2 -2.630247  3.260383    250 -0.032082
2     data1 -0.030280  0.614680    250  0.297251
      data2 -2.989741  2.458842    250  0.034358
3     data1  0.618510  3.927528    250  1.254808
      data2 -3.745356  2.954439    250  0.114908

### Example: Filling Missing Values with Group-Specific Values

When cleaning up missing data, in some cases you will remove data observations
using `dropna`, but in others you may want to **fill in the null `NA` values** using a
fixed value or some value derived from the data. 

`fillna` is the right tool to use; for
example, here I fill in the null values with the mean

In [104]:
s = pd.Series(np.random.standard_normal(6))
s

0   -0.159134
1   -0.417616
2    1.141487
3   -2.043018
4    0.125036
5    0.203608
dtype: float64

In [105]:
s[::2] = np.nan
s

0         NaN
1   -0.417616
2         NaN
3   -2.043018
4         NaN
5    0.203608
dtype: float64

In [106]:
s.fillna(s.mean())

0   -0.752342
1   -0.417616
2   -0.752342
3   -2.043018
4   -0.752342
5    0.203608
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the
data and use apply with a function that calls `fillna` on each data chunk. 

In [107]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio         -0.884727
New York     -0.026877
Vermont       0.441817
Florida       1.003998
Oregon        0.682841
Nevada        0.326045
California    0.425384
Idaho        -3.428254
dtype: float64

In [108]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.884727
New York     -0.026877
Vermont            NaN
Florida       1.003998
Oregon        0.682841
Nevada             NaN
California    0.425384
Idaho              NaN
dtype: float64

In [109]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [110]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [111]:
data.groupby(group_key).mean()

East    0.030798
West    0.554112
dtype: float64

In [112]:
#  fill the NA values using the group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.884727
      New York     -0.026877
      Vermont       0.030798
      Florida       1.003998
West  Oregon        0.682841
      Nevada        0.554112
      California    0.425384
      Idaho         0.554112
dtype: float64

In [113]:
# predefined fill values that vary by group. 
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

East  Ohio         -0.884727
      New York     -0.026877
      Vermont       0.500000
      Florida       1.003998
West  Oregon        0.682841
      Nevada       -1.000000
      California    0.425384
      Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation
Suppose you wanted to draw a **random sample** (with or without replacement) from a
large dataset for Monte Carlo simulation purposes or some other application. There
are a number of ways to perform the “draws”; here we use the sample method for
Series.

To demonstrate, here’s a way to construct a deck of English-style playing cards:

In [117]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

Now we have a Series of length 52 whose index contains card names, and values are
the ones used in blackjack and other games (to keep things simple, I let the ace "A"
be 1)

In [115]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [118]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

7D     7
JS    10
6H     6
8S     8
5H     5
dtype: int64

Suppose you wanted two random cards from each suit. Because the suit is the last
character of each card name, we can group based on this and use apply

In [119]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  9C    9
   3C    3
D  5D    5
   2D    2
H  3H    3
   7H    7
S  9S    9
   2S    2
dtype: int64

Alternatively, we could pass group_keys=False to drop the outer suit index, leaving
in just the selected cards

In [120]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C      9
5C      5
9D      9
6D      6
10H    10
4H      4
3S      3
JS     10
dtype: int64

### Example: Group Weighted Average and Correlation
Under the split-apply-combine paradigm of groupby, operations between columns in
a DataFrame or two Series, such as a **group weighted average**, are possible. As an
example, take this dataset containing group keys, values, and some weights

In [ ]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

In [ ]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

## Group Transforms and “Unwrapped” GroupBys
We looked at the `apply` method in grouped operations for performing transformations. There is
another built-in method called **`transform`**, which is similar to `apply` but imposes
**more constraints** on the kind of function you can use:
- It can produce a **scalar value** to be **broadcast** to the shape of the group.
- It can produce an object of the same shape as the input group.
- It must **not** **mutate** its input.


In [121]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [122]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

Suppose instead we wanted to produce a Series of the same shape as `df['value']` but
with **values replaced by the average grouped by 'key'**. We can pass a function that
computes the mean of a single group to transform

In [123]:
def get_mean(group):
    return group.mean()

g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

For built-in aggregation functions, we can pass a string alias as with the GroupBy agg
method

In [124]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

Like `apply`, `transform` works with functions that return Series, but the result must
be the same size as the input. For example, we can multiply each group by 2 using a
helper function:

In [125]:
def times_two(group):
    return group * 2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

Consider a group transformation function composed from simple aggregations

In [127]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [128]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [129]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

Built-in aggregate functions like `'mean'` or `'sum'` are often **much faster** than a general
`apply` function. These also have a “fast path” when used with `transform`. This allows
us to perform what is called an unwrapped group operation

In [130]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [131]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Here, we are doing arithmetic between the outputs of multiple GroupBy operations
instead of writing a function and passing it to groupby(...).`apply`. That is what is
meant by “unwrapped.”

While an unwrapped group operation may involve multiple group aggregations, the
overall benefit of vectorized operations often outweighs this

## Pivot Tables and Cross-Tabulation
A **pivot table** is a data **summarization** tool frequently found in spreadsheet programs
and other data analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectangle with some of the group keys along the rows and
some along the columns. 

Pivot tables in Python with `pandas` are made possible
through the `groupby` facility described in this chapter, **combined with** `reshape` operations utilizing `hierarchical indexing`. 

DataFrame also has a `pivot_table` method, and 
there is also a top-level `pandas.pivot_table` function. In addition to providing a
convenience interface to `groupby`, `pivot_table` can add partial totals, also known as
margins.

Returning to the tipping dataset, suppose you wanted to compute a table of group
means (the default pivot_table aggregation type) arranged by day and smoker on
the rows

In [132]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [133]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

This could have been produced with groupby directly, using `tips.groupby(["day","smoker"]).mean()`. 

Now, suppose we want to take the **average** of only `tip_pct` and
`size`, and additionally **group by time**. I’ll put smoker in the table columns and time
and day in the rows

In [134]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing `margins=True`. This
has the effect of adding `All` row and column labels, with corresponding values being
the **group statistics for all the data within a single tier**

In [135]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Here, the `All` values are **means** without taking into account smoker versus non-smoker (the `All` columns) or any of the two levels of grouping on the rows (the All
row).

To use an aggregation function other than mean, pass it to the `aggfunc` keyword
argument. 

For example, "count" or len will give you a cross-tabulation (count or
frequency) of group sizes (though "count" will exclude null values from the count
within data groups, while len will not):

In [136]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

If some combinations are empty (or otherwise `NA`), you may wish to pass a
`fill_value`

In [137]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

<img src="Img/pd_pivot_table.png" alt="pivot_table options" title="pivot_table options" />

### Cross-Tabulations: Crosstab
A **cross-tabulation** (or crosstab for short) is a special case of a pivot table that computes **group frequencies**.

In [138]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/var/folders/kw/6vcbxgjj1pj_yb81l3dd1zzw0000gn/T/ipykernel_60896/30910860.py:13: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_table(StringIO(data), sep="\s+")


In [139]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


As part of some survey analysis, we might want to **summarize** this data by nationality
and handedness. You could use `pivot_table` to do this, but the `pandas.crosstab`
function can be more convenient

In [140]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [141]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244